In [ ]:
import numpy as np

import load
import tensorflow as tf
import motion
import numpy as np
from motion import OpticalFlow
import tensorflow_compression as tfc
import tensorflow_addons as tfa

batch_size = 4
Height = 256
Width = 256
Channel = 3
lr_init = 1e-4
frames=2
I_QP = 27
l = 256


folder = np.load("/mnt/WindowsDev/Developer/tensorflow-wavelets/folder_cloud.npy")
#a = load.load_data()
data = np.zeros([frames, batch_size, Height, Width, Channel])
data = load.load_data(data, frames, batch_size, Height, Width, Channel, folder, I_QP)
print("Data Load done! ...")

In [ ]:
data.shape

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt


In [ ]:
cnt = 1
fig = plt.figure()
for idx in range(frames):
    for b in range(batch_size):
        ax = fig.add_subplot(frames, batch_size, cnt)
        imgplot = plt.imshow(data[idx, b, :, :, :].astype(np.uint8))
        plt.axis("off")
        cnt +=1
    

In [ ]:



num_filters = 128
kernel_size = 3
M = 2

inputs_y0_com = tf.keras.Input(shape=(Width, Height, Channel,))
inputs_y1_raw = tf.keras.Input(shape=(Width, Height, Channel,))

x = OpticalFlow()([inputs_y0_com, inputs_y1_raw])

mt = motion.encoder(x , num_filters=num_filters, kernel_size=kernel_size, M=M)

# Entropy bottelneck
noisy = tfc.NoisyNormal(loc=.5, scale=8.)
entropy_quantizer_mv = tfc.ContinuousBatchedEntropyModel(noisy, 1, compression=True)
mt_hat, MV_likelihoods = entropy_quantizer_mv(mt, training=True)
vt_hat = motion.decoder(mt_hat, num_filters=128, kernel_size=3, M=2)

model = tf.keras.Model([inputs_y0_com, inputs_y1_raw], vt_hat)
# Y1_warp = tfa.image.dense_image_warp(inputs_y0_com, vt_hat )
# MC_input = tf.concat([vt_hat, inputs_y0_com, Y1_warp], axis=-1)
# Y1_MC = motion.MotionCompensation(MC_input)
# Res = inputs_y1_raw - Y1_MC
# res_latent = motion.encoder(Res, num_filters=128, kernel_size=5, M=128)
# 


In [ ]:
epochs = 15
(x_train, y_train), (x_test, y_test) = load_mnist(remove_n_samples=1000)

model.compile(optimizer="rmsprop", loss="categorical_crossentropy")
model.fit([x_train, x_train], x_train, validation_split=0.2, epochs=20, batch_size=32, verbose=1,)

# model.fit(train_gen, epochs=epochs, validation_data=val_gen, callbacks=callbacks)

In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
def load_mnist(categorical=True, remove_n_samples=1000, expand_d=False):
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    # Remove images to get smaller dataset
    if remove_n_samples != 0:
        x_train = x_train[:remove_n_samples, :, :]
        y_train = y_train[:remove_n_samples]
        x_test = x_test[:remove_n_samples//2, :, :]
        y_test = y_test[:remove_n_samples//2]

    if categorical:
        y_train = to_categorical(y_train)
        y_test = to_categorical(y_test)

    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0

    x_train = np.expand_dims(x_train, axis=-1)
    x_test = np.expand_dims(x_test, axis=-1)

    x_train = np.repeat(x_train, 3, axis=-1)
    x_test = np.repeat(x_test, 3, axis=-1)

    x_train = tf.image.resize(x_train, [256,256]) # if we want to resize
    x_test = tf.image.resize(x_test, [256,256]) # if we want to resize

    if expand_d:
        x_train = np.expand_dims(x_train, axis=-1)
        x_test = np.expand_dims(x_test, axis=-1)

    return (x_train, y_train), (x_test, y_test)

In [ ]:
(x_train, y_train), (x_test, y_test) = load_mnist(remove_n_samples=1000)

In [ ]:
import numpy as np

import load
import tensorflow as tf
import motion
import numpy as np
from motion import OpticalFlow
import tensorflow_compression as tfc
import tensorflow_addons as tfa

batch_size = 4
Height = 256
Width = 256
Channel = 3
lr_init = 1e-4
frames=2
I_QP = 27
l = 256


folder = np.load("/mnt/WindowsDev/Developer/tensorflow-wavelets/folder_cloud.npy")
#a = load.load_data()
data = np.zeros([frames, batch_size, Height, Width, Channel])
data = load.load_data(data, frames, batch_size, Height, Width, Channel, folder, I_QP)
print("Data Load done! ...")

import tensorflow as tf
from motion import OpticalFlow


inputs_y0_com = tf.keras.Input(shape=(Width, Height, Channel,))
inputs_y1_raw = tf.keras.Input(shape=(Width, Height, Channel,))

x = OpticalFlow()([inputs_y0_com, inputs_y1_raw])

model = tf.keras.Model([inputs_y0_com, inputs_y1_raw], x)

model.compile(optimizer="rmsprop", loss="categorical_crossentropy")
model.fit([data[0], data[1]], data[0], validation_split=0.2, epochs=20, batch_size=32, verbose=1,)






In [ ]:
import tensorflow as tf
import numpy as np
import tensorflow_addons as tfa
import matplotlib.pyplot as plt

img_path = tf.keras.utils.get_file('tensorflow.png','https://tensorflow.org/images/tf_logo.png')
img_raw = tf.io.read_file(img_path)
img = tf.io.decode_image(img_raw)
img = tf.image.convert_image_dtype(img, tf.float32)
img = tf.image.resize(img, [500,500])

plt.title("TensorFlow Logo with shape {}".format(img.shape))
_ = plt.imshow(img)




In [ ]:
input_img = tf.image.convert_image_dtype(tf.expand_dims(img, 0), tf.dtypes.float32)
print(input_img.shape)
flow_shape = [1, input_img.shape[1], input_img.shape[2], 2]
init_flows = np.float32(np.random.normal(size=flow_shape) *1.0)
print(init_flows.shape)

dense_img_warp = tfa.image.dense_image_warp(input_img, init_flows)
dense_img_warp = tf.squeeze(dense_img_warp, 0)
_ = plt.imshow(dense_img_warp)

In [5]:
import numpy as np
from tensorflow.python.keras.engine import training

import load
import tensorflow as tf
import motion
import numpy as np
import tensorflow_compression as tfc
import tensorflow_addons as tfa

# tf.config.run_functions_eagerly(True)

batch_size = 4
Height = 256
Width = 256
Channel = 3
lr_init = 1e-4
frames=2
I_QP = 27
l = 256


folder = np.load("/mnt/WindowsDev/Developer/tensorflow-wavelets/folder_cloud.npy")
#a = load.load_data()
data = np.zeros([frames, batch_size, Height, Width, Channel])
data = load.load_data(data, frames, batch_size, Height, Width, Channel, folder, I_QP)
print("Data Load done! ...")

noisy = tfc.NoisyNormal(loc=.5, scale=8.)
entropy_quantizer_mv = tfc.ContinuousBatchedEntropyModel(noisy, 1, compression=True)

inputs_y0_com = tf.keras.Input(shape=(Width, Height, Channel,))
inputs_y1_raw = tf.keras.Input(shape=(Width, Height, Channel,))
flow_4 = motion.optical_flow(inputs_y0_com, inputs_y1_raw, batch_size, Height, Width)
mt = motion.encoder(flow_4, num_filters=128, kernel_size=3, M=128)
mt_hat, MV_likelihoods = entropy_quantizer_mv(mt, training=True)
vt_hat = motion.decoder(mt_hat, num_filters=128, kernel_size=3, M=2)
Y1_warp = tf.keras.layers.Lambda(lambda a: tfa.image.dense_image_warp(a[0], a[1]))((inputs_y0_com, vt_hat))

# motion compenstation
MC_input = tf.concat([vt_hat, inputs_y0_com, Y1_warp], axis=-1)
# shape (1, 240 416, 8)
Y1_MC = motion.MotionCompensation(MC_input)

Res = inputs_y1_raw - Y1_MC


model = tf.keras.Model([inputs_y0_com, inputs_y1_raw], Y1_warp)

model.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError())




Data Load done! ...


In [6]:
model.fit([data[0], data[1]], data[0],
                epochs=5,
                batch_size=4,)

/home/ubu-admin/anaconda3/envs/py38/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py:3703: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  warnings.warn(


Epoch 1/5
1/1 [==============================] - 72s 72s/step - loss: 6.1838
Epoch 2/5
1/1 [==============================] - 17s 17s/step - loss: 4.3066
Epoch 3/5
1/1 [==============================] - 18s 18s/step - loss: 4.3146
Epoch 4/5
1/1 [==============================] - 17s 17s/step - loss: 3.3611
Epoch 5/5
1/1 [==============================] - 18s 18s/step - loss: 4.5714
